# Fetal Brain Tractography Pipeline Tutorial

This tutorial walks through a pipeline for generating tractography in fetal brains using diffusion tensor imaging (DTI). The pipeline covers several steps: from converting the tensor to an Orientation Distribution Function (ODF), to adjusting parcellations, creating a 5-tissue type (5TT) image, and generating tractography for further analysis.

## Pipeline Overview

1. Convert the diffusion tensor image to an Orientation Distribution Function (ODF).
2. Use a tissue segmentation image and a parcellation image to create necessary brain masks and adjust parcellations.
3. Generate a 5-tissue type (5TT) image from the tissue segmentation.
4. Adjust the parcellation to ensure white matter regions match the tissue segmentation.
5. Separate gray and white matter regions for tractography extraction.
6. Generate the gray matter-white matter interface (GMWMI) for tractography seeding.
7. Use the whole brain mask to set tract length constraints.
8. Generate tractography using the ODF, GMWMI, and the 5TT image.
9. Convert the tractography to a format compatible with tract extraction.
10. Extract specific tracts from the tractography.


## Requirements
- MRTrix (https://mrtrix.readthedocs.io/en/3.0.4/index.html)
- tract_querier (https://tract-querier.readthedocs.io/en/latest/)
    - Should be installed in in a python 2.7 environment

## Python Packages
- ants
- DiPy
- pandas
- nibabel
- numpy
- SimpleITK
- scipy

In [24]:
import os
import nibabel as nib
import subprocess

#Get current working directory and file paths
current_dir = os.getcwd()

## Step 1: Convert Tensor to ODF

In this step, we convert the diffusion tensor data into an ODF (Orientation Distribution Function). This step is necessary for generating tractography using the iFOD2 algorithm.

In [25]:
from tensor2odf import compute_fod_sh

# Load the diffusion tensor data
tensor_path = os.path.join(current_dir, 'example', 'GA23.nii.gz')
tensor = nib.load(tensor_path)
tensor_data = tensor.get_fdata()

# Compute the FOD (Fiber Orientation Distribution)
fod_path = os.path.join(current_dir, 'example', 'GA23_odf.nii.gz')
fod_data = compute_fod_sh(tensor_data)
fod_img = nib.Nifti1Image(fod_data, tensor.affine)
nib.save(fod_img, fod_path)

## Step 2: Tissue Segmentation and Parcellation (Prerequisite)

This step assumes you have already obtained a tissue segmentation image and a parcellation image. These segmentations are typically generated via atlas-based segmentation or a deep learning model (e.g., Davood). The segmentation is not included in this pipeline, but they are necessary inputs for the following steps.

## Step 3: Create a 5-Tissue Type (5TT) Image and Whole Brain Mask

This step converts the tissue segmentation image into a 5TT image, which differentiates cortical gray matter, sub-cortical gray matter, white matter, cerebrospinal fluid (CSF), and pathological tissues. A whole brain mask is also created that can be used for tractography seeding. These step requires having a .csv file with the mapping of the different tissues. The label key for the CRL tissue segmentations can be found in 5tt_key.csv.


In [26]:
from fivett_gen import fivett_segmentation

# Load the tissue segmentation data
tissue_path = os.path.join(current_dir, 'example', 'GA23_tissue.nii.gz')
md_path = os.path.join(current_dir, 'example', 'GA23_md.nii.gz')
fivett_key_path = os.path.join(current_dir, '5tt_key.csv')

tissue = nib.load(tissue_path)
tissue_data = tissue.get_fdata()

md = nib.load(md_path)
md_data = md.get_fdata()

# Generate 5TT image
_5tt_img_path = os.path.join(current_dir, 'example', 'GA23_5tt.nii.gz')
five_tt_img_data, whole_brain_data, csf_mask_data = fivett_segmentation(tissue_data, md_data, fivett_key_path, tissue.header.get_zooms()[0])
five_tt_img = nib.Nifti1Image(five_tt_img_data, tissue.affine)
nib.save(five_tt_img, _5tt_img_path)

## Step 4: Adjust Parcellation to Match the Tissue Segmentation

This step ensures that the white matter regions in the parcellation imnage align perfectly with those in the tissue segmentation. Conflicting regions between the two segmentations are removed, and watershed dilation is applied to refine the boundaries. This step is necessary for accurate tract extraction after tractography generation. Please note that these will only work for the tissue segmentations of the CRL and the parcellation scheme based on the ENA55 Atlas. 

In [27]:
from adjust_parcellation import adjust_parcellation

parcellation_path = os.path.join(current_dir, 'example', 'GA23_regional.nii.gz')
parcellation_adjusted_path = os.path.join(current_dir, 'example', 'GA23_regional_adjusted.nii.gz')

# Load the parcellation data
parcellation = nib.load(parcellation_path)
parcellation_data = parcellation.get_fdata()

# Adjust the parcellation to align with tissue segmentation
parcellation_adjusted_data = adjust_parcellation(tissue_data, parcellation_data)
parcellation_adjusted_img = nib.Nifti1Image(parcellation_adjusted_data, parcellation.affine)
nib.save(parcellation_adjusted_img, parcellation_adjusted_path)

## Step 5: Separate Gray Matter and White Matter in Parcellation

In this step, we divide the cortical parcellations into their respective gray and white matter regions. This ensures that the original ENA50 parcellations follow the FreeSurfer parcellation scheme. This step is crucial for the tract extraction step using tract_querier and the provided queries.

In [28]:
from create_ctx_wm import parcellation_ctx_wm

voxel_size = tissue.header.get_zooms()[0]

ctx_wm_parcellation_data = parcellation_adjusted_data.copy() # Should be used because the parcellation data is modified in the function

# Create the cortical gray matter and white matter parcellation
ctx_wm_parcellation_path = os.path.join(current_dir, 'example', 'GA23_ctx_wm.nii.gz')
ctx_wm_parcellation_data = parcellation_ctx_wm(tissue_data, ctx_wm_parcellation_data, voxel_size)
ctx_wm_parcellation_img = nib.Nifti1Image(ctx_wm_parcellation_data, tissue.affine)
nib.save(ctx_wm_parcellation_img, ctx_wm_parcellation_path)

## Step 6: Generate Gray Matter-White Matter Interface (GMWMI)

The 5TT image is used to generate the gray matter-white matter interface (GMWMI), which is used for seeding tractography. The GMWMI is the boundary between the gray and white matter regions. For the CRL data, using this type of seeding produces the best results. This image is calculated using the 5TT image and the command can be found in MRTrix.

In [29]:
# Generate GMWMI from 5TT image
gmwmi_img_path = os.path.join(current_dir, 'example', 'GA23_gmwmi.nii.gz')
_5tt_img_path = os.path.join(current_dir, 'example', 'GA23_5tt.nii.gz')

command = ['5tt2gmwmi', _5tt_img_path, gmwmi_img_path, '-force']
subprocess.run(command, check=True)

5tt2gmwmi: [WARNING] existing output files will be overwritten
5tt2gmwmi: [100%] uncompressing image "/Users/camilocalixto/Dropbox/BCH/Jobs/Tractography_pipeline/example/GA23_5tt.nii.gz"[0K
5tt2gmwmi: [100%] Generating GMWMI seed mask[0K
5tt2gmwmi: [100%] compressing image "/Users/camilocalixto/Dropbox/BCH/Jobs/Tractography_pipeline/example/GA23_gmwmi.nii.gz"[0K


CompletedProcess(args=['5tt2gmwmi', '/Users/camilocalixto/Dropbox/BCH/Jobs/Tractography_pipeline/example/GA23_5tt.nii.gz', '/Users/camilocalixto/Dropbox/BCH/Jobs/Tractography_pipeline/example/GA23_gmwmi.nii.gz', '-force'], returncode=0)

## Step 7: Calculate Minimum and Maximum Streamline Lengths

To ensure reliable tractography, calculate the minimum and maximum lengths of the streamlines using the whole brain mask. This is important to filter out overly long streamlines (erroneous) or very short streamlines (U-fibers) that don't contribute to major tracts. If the tractography is to be used for structural connectivity, the last argument of the calculate ``calculate_streamline_lengths`` function should be changed to 'connectivity'. 


In [21]:
from get_max_min_streamline import calculate_streamline_lengths

# Calculate streamline lengths
streamline_lengths = calculate_streamline_lengths(whole_brain_data, tissue.header.get_zooms(), 'tract')
minlength = streamline_lengths[0]
maxlength = streamline_lengths[1]

print(f'Minimum streamline length: {minlength}')
print(f'Maximum streamline length: {maxlength}')

Minimum streamline length: 24
Maximum streamline length: 78


## Step 8: Generate Tractography

We generate tractography using the ODF, GMWMI, and 5TT images with MRTrix’s tckgen command. 

For crl fetal data, and when the tensor is converted to an ODF, the optimal parameters for the iFOD2 algorithm are:  -cutoff 0.001, -power 10, -angle 15, 20 or 25. For a list of the best angles for each tract, please refer to tract_angles.csv file.

In [8]:
odf_image = os.path.join(current_dir, 'example', 'GA23_odf.nii.gz')
tractography_output = os.path.join(current_dir, 'example', 'GA23.tck')

act_image = _5tt_img_path
seed_image = gmwmi_img_path

# For CRL tensor, the threshold is 0.001, power of 10, angle of 15, 20 or 25
streamlines_number = 10000
power = 10
angle = 15
threshold = 0.001  

command = ['tckgen', odf_image, tractography_output, '-act', act_image, '-seed_gmwmi', seed_image, '-cutoff', str(threshold), '-select', str(streamlines_number), '-power', str(power), '-angle', str(angle), '-minlength', str(minlength), '-maxlength', str(maxlength), '-force']
subprocess.run(command, check=True)

tckgen: [WARNING] existing output files will be overwritten
tckgen: [100%] uncompressing image "/Users/camilocalixto/Dropbox/BCH/Jobs/Tractography_pipeline/example/GA23_5tt.nii.gz"[0K
tckgen: [100%] uncompressing image "/Users/camilocalixto/Dropbox/BCH/Jobs/Tractography_pipeline/example/GA23_gmwmi.nii.gz"[0K
tckgen: [100%] uncompressing image "/Users/camilocalixto/Dropbox/BCH/Jobs/Tractography_pipeline/example/GA23_odf.nii.gz"[0K
tckgen: [100%] preloading data for "/Users/camilocalixto/Dropbox/BCH/Jobs/Tractography_pipeline/example/GA23_odf.nii.gz"[0K
tckgen: [100%] uncompressing image "/Users/camilocalixto/Dropbox/BCH/Jobs/Tractography_pipeline/example/GA23_5tt.nii.gz"[0K
tckgen: [100%]    91809 seeds,    91809 streamlines,    10000 selected


CompletedProcess(args=['tckgen', '/Users/camilocalixto/Dropbox/BCH/Jobs/Tractography_pipeline/example/GA23_odf.nii.gz', '/Users/camilocalixto/Dropbox/BCH/Jobs/Tractography_pipeline/example/GA23.tck', '-act', '/Users/camilocalixto/Dropbox/BCH/Jobs/Tractography_pipeline/example/GA23_5tt.nii.gz', '-seed_gmwmi', '/Users/camilocalixto/Dropbox/BCH/Jobs/Tractography_pipeline/example/GA23_gmwmi.nii.gz', '-cutoff', '0.001', '-select', '10000', '-power', '10', '-angle', '15', '-minlength', '24', '-maxlength', '78', '-force'], returncode=0)

## Step 9: Convert TCK to TRK Format

Convert the tractography from .tck format to .trk format, which is required for tract extraction and visulization in TrackVis.

In [9]:
from dipy.io.streamline import load_tractogram, save_tractogram

def tck2trk(tck_file, anatomy_ref):
    whole_brain = load_tractogram(tck_file, anatomy_ref)
    trk_file = tck_file.replace(".tck", ".trk")
    save_tractogram(whole_brain, trk_file)
    return trk_file

trk_file_path = tck2trk(tractography_output, md)

## Step 10: Extract Specific Tracts

Use tract_querier from the WMQL to extract specific tracts from the tractography based on an anatomical query file. The query file "crl_fetal_parcellation_ACT.qry" and "crl_fetal_parcellation_NON_ACT.qry" can be used to extract tracts from segmentations based on the CRL tissue segmentations and the ENA55 atlas cortical parcellations. The main difference between the two query files is that the ACT query file is used for tractography generated using the Anatomically Constrained Tractography (ACT) algorithm, while the NON_ACT query file is used for tractography generated without ACT.

Please note that steps 4 and 5 are crucial for this to work, and that if another another parcellation scheme is used, this query file would not work. 

In [22]:
tract_extraction_output = os.path.join(current_dir, 'example', 'tract_extraction', 'GA23')
ctx_wm_parcellation_path = os.path.join(current_dir, 'example', 'GA23_ctx_wm.nii.gz')
query_file = os.path.join(current_dir, 'crl_fetal_parcellation_ACT.qry')

command = f"tract_querier -t {trk_file_path} -a {ctx_wm_parcellation_path} -q {ctx_wm_parcellation_path} -o {tract_extraction_output}"
subprocess.run(command, shell=True)

/bin/sh: tract_querier: command not found


CompletedProcess(args='tract_querier -t /Users/camilocalixto/Dropbox/BCH/Jobs/Tractography_pipeline/example/GA23.trk -a /Users/camilocalixto/Dropbox/BCH/Jobs/Tractography_pipeline/example/GA23_ctx_wm.nii.gz -q /Users/camilocalixto/Dropbox/BCH/Jobs/Tractography_pipeline/example/GA23_ctx_wm.nii.gz -o /Users/camilocalixto/Dropbox/BCH/Jobs/Tractography_pipeline/example/tract_extraction/GA23', returncode=127)

## Step 11 (optional): Extract Brain Structures

If you want to extract brain structures from a segmentation/parcellation image, you can use the extract_individual_labels function. This function extracts individual labels from a segmentation image creating a binary mask for each label. This mask can then be used to visualize the brain structures in 3D or can be used as seeds for tractography. A .csv file with the mapping between the label values and the structure names is required. Here we use the labels.csv file to map the label values to the structure names.

In [30]:
from extract_labels import extract_individual_labels

labelkey_path = os.path.join(current_dir, 'labelkey.csv')
output_dir = os.path.join(current_dir, 'example', 'structure_extraction')

extract_individual_labels(parcellation_adjusted_data, parcellation_adjusted_img.affine, 
labelkey_path, output_dir, base_name="GA23")

Segmentation file created for: Precentral_L
Segmentation file created for: Precentral_R
Segmentation file created for: Frontal_Sup_L
Segmentation file created for: Frontal_Sup_R
Segmentation file created for: Frontal_Sup_Orb_L
Segmentation file created for: Frontal_Sup_Orb_R
Segmentation file created for: Frontal_Mid_L
Segmentation file created for: Frontal_Mid_R
Segmentation file created for: Frontal_Mid_Orb_L
Segmentation file created for: Frontal_Mid_Orb_R
Segmentation file created for: Frontal_Inf_Oper_L
Segmentation file created for: Frontal_Inf_Oper_R
Segmentation file created for: Frontal_Inf_Tri_L
Segmentation file created for: Frontal_Inf_Tri_R
Segmentation file created for: Frontal_Inf_Orb_L
Segmentation file created for: Frontal_Inf_Orb_R
Segmentation file created for: Rolandic_Oper_L
Segmentation file created for: Rolandic_Oper_R
Segmentation file created for: Supp_Motor_Area_L
Segmentation file created for: Supp_Motor_Area_R
Segmentation file created for: Olfactory_L
Segme